## merge results
For now, mrcnn model offers very good results with -1,  
yet unet offers correct mask results.  
So we want to merge them.  


In [2]:
import pandas as pd


In [7]:
# load results
mrcnn_df = pd.read_csv('./mrcnn.csv')
unet_df = pd.read_csv('./unet.csv')
sample_df = pd.read_csv('./')

In [10]:
# get neg id from mrcnn result
neg_df = mrcnn_df[mrcnn_df['EncodedPixels']==' -1']
neg_id_list = list(neg_df['ImageId'])

print(len(neg_id_list))

323


In [18]:
# get pos id from unet result
pos_df = unet_df[unet_df['EncodedPixels']!=' -1']
pos_id_list = list(set(pos_df['ImageId']))

print(len(pos_id_list))

1377


In [19]:
unique_id_list = list(set(unet_df['ImageId']))

# get pos id from unet result
pos_id_list = []

for uid in unique_id_list:
    if uid not in neg_id_list:
        pos_id_list.append(uid)

print(len(pos_id_list), len(unique_id_list))

1054 1377


In [20]:
# export all empty csv
ulist = []

for uid in unique_id_list:
    ulist.append([uid, ' -1'])
    
udf = pd.DataFrame(ulist, columns=['ImageId', 'EncodedPixels'])
udf.to_csv('empty.csv', index=False)

In [15]:
# now, merge
merge_list = []

for neg_id in neg_id_list:
    merge_list.append([neg_id, ' -1'])
    
for pos_id in pos_id_list:
    pos_df = unet_df[unet_df['ImageId']==pos_id]
    rle_list = list(pos_df['EncodedPixels'])
    
    for rle in rle_list:
        merge_list.append([pos_id, rle])
        

In [17]:
# export
merge_df = pd.DataFrame(merge_list, columns=['ImageId', 'EncodedPixels'])
merge_df.to_csv('merge.csv', index=False)
